# Organoid Contour segmentation

This notebook implements the organoid contour segmentation process that goes along with Phindr3d.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.ndimage as nd
import skimage.io as io
import metadata_functions as mf
import phindr_organoidCSApp as cs
import phindr_functions as ph
import tifffile as tif
from IPython.display import display

# Segmentation settings (Not user modifiable in matlab)

In [ ]:
# parameters
min_area_spheroid = 200 # Minimum Area
intensity_threshold = 500 # Minimum Intensity (Average MIP Intensity)
radius_spheroid = 75 #Approx Radius in pixels
smoothin_param=0.01 #Smoothing Factor
scale_spheroid =1 #Scale Factor
entropy_threshold = 1 

# Create metadata

In [ ]:
# metadatafilename = 'metadata_python.txt'
# folder_path = r'E:\Phindr3D_organoid-sample-data'
# regex = '(?P<WellID>\w+)f(?P<Field>\w+)p(?P<Stack>\d+)-ch(?P<Channel>\d)sk1fk1fl1.tiff'

# mdata_path = mf.createMetadata(folder_path, regex)

# directory and metadata set-up

In [ ]:
mdata_path = r'E:\Phindr3D_organoid-sample-data\metadata_python.txt'
mdata = pd.read_csv(mdata_path, sep='\t')
header = mdata.columns
channels = header[header.map(lambda head: head.startswith('Channel_'))]
imageID = header[header == 'ImageID']
filenameData = header[header.map(lambda head: (head not in channels) and (head not in imageID) and (head != 'MetadataFile') and (head != 'Stack'))]
numChannels = len(channels)
uImageID = np.unique(mdata[imageID])


mdata

In [ ]:
# select output directory:
outputfolder = r'C:\Users\teole\anaconda3\envs\phy479\mdatatestingsubset\segmentationoutput'
labpath = os.path.join(outputfolder, 'LabelledImages')
segpath = os.path.join(outputfolder, 'SegmentedImages')
# os.makedirs(labpath)
# os.makedirs(segpath)

In [ ]:
# select channel for segmentation:
segChannel = 0 # integer (0 to numChannels - 1) 
channelForSegmentation = channels[segChannel]

# Running segmentation and saving images

In [ ]:
for id in uImageID[-1:]:
    imageData = mdata.loc[mdata['ImageID']==id]
    zVals = np.unique(imageData['Stack']) #list of z planes

    IM, focusIndex = cs.getfsimage(imageData, channelForSegmentation)
    seg_unused, L = cs.getSegmentedOverlayImage(IM, min_area_spheroid, radius_spheroid, smoothin_param, entropy_threshold, intensity_threshold, scale_spheroid)
    
    uLabels = np.unique(L)
    uLabels = uLabels[uLabels != 0]
    numObjects = len(uLabels)
    if numObjects == 0:
        print('No objects found')
    ll = np.zeros((numObjects, 2), dtype=int)
    for iObjects in range(numObjects):
        nL = (L == uLabels[iObjects])
        ll[iObjects, :] = cs.getFocusplanesPerObjectMod(nL, focusIndex, numZ=np.sum(mdata['ImageID']==id)) #ll is full of array indices
    fstruct = nd.find_objects(L.astype(int))
    for iObjects in range(numObjects):
        for iPlanes in range(ll[iObjects, 0], ll[iObjects, 1]+1): #iPlanes also gets the indices. (but we want up-to and including ll[i, 1])
            for kChannels in range(numChannels):
                IM1 = io.imread( imageData.loc[imageData['Stack']==iPlanes, channels[kChannels]].values[0] )
                IM2 = IM1[fstruct[iObjects]]  ############### This may be on the small side by 1 pixel in each dimension. may need to edit the slices out by 1.
                filenameParts = []
                for dfcol in filenameData:
                    part = f'{dfcol[0]}{imageData.loc[imageData["Stack"]==zVals[0], dfcol].values[0]}'
                    filenameParts.append(part)
                filenameParts.append(f'Z{iPlanes}')
                filenameParts.append(f'CH{kChannels+1}')
                filenameParts.append(f'ID{id}')
                filenameParts.append(f'OB{iObjects+1}')
                obFileName = '__'.join(filenameParts)
                obFileName = obFileName + '.tiff'
                tif.imwrite(os.path.join(segpath, obFileName), IM2)
        filenameParts = []
        for dfcol in filenameData:
            part = f'{dfcol[0]}{imageData.loc[imageData["Stack"]==zVals[0], dfcol].values[0]}'
            filenameParts.append(part)
        filenameParts.append('Z1')
        filenameParts.append(f'CH{kChannels+1}')
        filenameParts.append(f'ID{id}')
        filenameParts.append(f'OB{iObjects+1}')
        obFileName = '__'.join(filenameParts)
        obFileName = obFileName + '.tiff'
        IML = L[fstruct[iObjects]]
        tif.imwrite(os.path.join(labpath, obFileName), IML)
    filenameParts = []
    for dfcol in filenameData:
        part = f'{dfcol[0]}{imageData.loc[imageData["Stack"]==zVals[0], dfcol].values[0]}'
        filenameParts.append(part)
    filenameParts.append(f'ID{id}')
    filenameParts.append(f'ALLObjects')
    obFileName = '__'.join(filenameParts)
    obFileName = obFileName + '.tiff'
    IML = L
    tif.imwrite(os.path.join(labpath, obFileName), IML)

print('\nAll Done!\n')
